In [83]:
!pip install haversine

import haversine as hs
import pickle
import numpy as np
import requests
import json
import pandas as pd

from numpy import asarray

In [75]:
# importing station df
url = "station_df.csv"
station_df = pd.read_csv(url)

# importing mall df
url = "Mall_df.csv"
mall_df = pd.read_csv(url)

In [102]:
# input address and extracting coordinates data and sector_code from onemap api

address = input("input address here:")
remaining_months = int(input("input remaining months here:"))
storey = int(input("HDB Flat storey here:"))


url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal="+address+"&returnGeom=Y&getAddrDetails=Y&pageNum=1"

response = requests.request("GET", url)

query = json.loads(response.text)
results = query["results"]
latitude = results[0]["LATITUDE"]
longitude = results[0]["LONGITUDE"]
sector_code = results[0]["POSTAL"][:2]

address_coordinate = (float(latitude), float(longitude))
print(address_coordinate)

# print(latitude)
# print(longitude)
# print(sector_code)


input address here:680 WOODLANDS AVE 6
input remaining months here:937
HDB Flat storey here:8
(1.43963261801972, 103.802121646793)


In [103]:
# nearest MRT distance

stationLatitudeList = list(station_df['latitude'])
stationLongitudeList = list(station_df['longitude'])
stationList = list(station_df['station'])

def nearestMRT(coordinate):
    min = 99999999
    station = ""
    for i, b in enumerate(stationLatitudeList):
        distance = hs.haversine(coordinate, (float(stationLatitudeList[i]), float(stationLongitudeList[i])))
        if (min > distance):
            min = distance
            station = stationList[i]
    return min

nearest_station_distance = nearestMRT(address_coordinate)
print(nearest_station_distance)

# distance from CBD
distance_from_CBD = hs.haversine(address_coordinate, (1.28393326234538, 103.851463066212))
print(distance_from_CBD)

# nearest Mall distance
mallLatitudeList = list(mall_df['latitude'])
mallLongitudeList = list(mall_df['longitude'])
mallList = list(mall_df['mall'])

def nearestMall(coordinate):
    min = 99999999
    mall = ""
    for i, b in enumerate(mallList):
        distance = hs.haversine(coordinate, (float(mallLatitudeList[i]), float(mallLongitudeList[i])))
        if (min > distance):
            min = distance
            mall = mallList[i]
    return min

nearest_mall_distance = nearestMall(address_coordinate)
print(nearest_mall_distance)

0.16464640099314404
18.161083833119896
0.10872986014273653


In [104]:
# extracting cluster coordinates data from json to compare

with open("cluster_coordinates.json", "r") as infile:
    data = json.load(infile)

cluster_json = json.loads(data)
sector_cluster_data = cluster_json[str(sector_code)]

target_coordinate = [float(latitude), float(longitude)]

# find the closest cluster
closest_cluster = None
min_distance = float('inf')

for cluster_name, cluster_coordinates in sector_cluster_data.items():
    cluster_x = float(cluster_coordinates[0])
    cluster_y = float(cluster_coordinates[1])
    distance = ((target_coordinate[0] - cluster_x) ** 2 + (target_coordinate[1] - cluster_y) ** 2) ** 0.5
    
    if distance < min_distance:
        min_distance = distance
        closest_cluster = cluster_name

print(closest_cluster)
    


cluster4


In [105]:
# open the correct pickle file for XGBoost algorithm

cluster_num = closest_cluster.split("cluster")[1]

pickle_file_name = "sector_code_" + sector_code + "_cluster_" + cluster_num + "_model.pkl"

with open(pickle_file_name, 'rb') as f:
    loaded_model = pickle.load(f)


In [106]:
X_new = np.array([remaining_months, storey, nearest_station_distance, distance_from_CBD, nearest_mall_distance])
X_new = X_new.reshape(1, -1)
y_pred = loaded_model.predict(X_new)
print(y_pred)

[4851.4424]
